<a href="https://colab.research.google.com/github/Shadoww002/PyTorch-Learning/blob/main/Chapter%203%20%E2%80%93%20Computer%20Vision/ComputerVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## IMPORT LIBRARIES
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [ ]:
## Getting Started With FashinMINST Datasets
train_data = datasets.FashionMNIST(
    root="Data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

test_data = datasets.FashionMNIST(
    root="Data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor(),
    target_transform=None
)

In [ ]:
len(train_data) , len(test_data)

In [ ]:
train_data.data[0].shape

In [ ]:
## Lets see The data

train_data[0] , train_data.test_labels[0]

In [ ]:
class_to_idx = train_data.class_to_idx
class_to_idx

In [ ]:
train_data.targets

In [ ]:
image , lable = train_data[0]
image.size()

In [ ]:
class_name = train_data.classes
class_name

In [ ]:
## Visualise The Data
plt.title(class_name[lable])
plt.imshow(image.squeeze())

In [ ]:
plt.title(class_name[lable])
plt.imshow(image.squeeze() ,
           cmap = "gray")

In [ ]:
## Ploting The Random Images
torch.manual_seed(42)
plt.figure(figsize=(8,8))
rows , cols = 4 , 4

for i in range(1 , rows*cols+1):
  idx = torch.randint(0,len(train_data),size=[1])
  image , label = train_data[idx.squeeze()]
  class_name = train_data.classes[label]
  plt.subplot(rows , cols , i)
  plt.title(class_name)
  plt.imshow(image.squeeze(), cmap="gray")
  plt.axis(False)


In [ ]:
torch.utils

In [ ]:
##Preparing DataLoader
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset=train_data,
                             batch_size = 32,
                             shuffle = True)
test_dataloader = DataLoader(dataset=test_data,
                             batch_size = 32,
                             shuffle = False)

In [ ]:
len(train_dataloader) , len(test_dataloader)

In [ ]:
60000/32 , 10000/32

In [ ]:
## Interact with the Data Loader
train_features_batch , train_labels_batch = next(iter(train_dataloader))
train_features_batch.shape , train_labels_batch.shape

In [ ]:
class_name = train_data.classes

In [ ]:
## Show a Sample
random_idx = torch.randint(0 , len(train_features_batch) , size=[1]).item()
img , label = train_features_batch[random_idx] , train_labels_batch[random_idx]
plt.title(class_name[label])
plt.imshow(img.squeeze(),cmap="gray")
plt.axis(False)

In [ ]:
## Understnding the Flatten Layer Concept
flatten_model = nn.Flatten()

x = train_features_batch[0]

output = flatten_model(x)

x.shape , output.shape

In [ ]:
## Creating a BaselIne Model
from torch import nn
class FashionMNISTModelV0(nn.Module):
  def __init__(self,input_shape ,hidden_units, output_shape):
    super().__init__()
    self.LayerStack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = input_shape,
                  out_features=hidden_units),
        nn.Linear(in_features=hidden_units,
                  out_features = output_shape)
    )

  def forward(self , x):
    return self.LayerStack(x)

In [ ]:
28*28

In [ ]:
torch.manual_seed(42)
model_0 = FashionMNISTModelV0(
    input_shape=28*28,
    hidden_units=10,
    output_shape=len(class_name)
).to("cpu")

model_0

In [ ]:
##Import HELPER Functions

import requests
from pathlib import Path

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
# Import accuracy metric
from helper_functions import accuracy_fn # Note: could also use torchmetrics.Accuracy(task = 'multiclass', num_classes=len(class_names)).to(device)

# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss() # this is also called "criterion"/"cost function" in some places
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.1)

In [ ]:
## Creating a Function To Time Experiments
from timeit import default_timer as timer

def print_train_time (
    start : float ,
    end : float,
    device : torch.device = None):

  total_time = end - start
  print(f"Train Time : {total_time:.3f} Sec on {device}")

  return total_time

In [ ]:
len(train_dataloader.dataset)

In [ ]:
## Building a Train and Test loop

# import tqdm
from tqdm.auto import tqdm

#setting seed and start time
torch.manual_seed(42)
train_start_time_cpu = timer()

# Set epochs
epochs = 3

##creating a loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch : {epoch}\n------")
  ##Train Loop
  train_loss = 0
  for batch , (X,y) in enumerate(train_dataloader):
    model_0.train()

    y_pred = model_0(X)

    loss = loss_fn(y_pred , y)
    train_loss += loss

    optimizer.zero_grad()

    loss.backward()

    optimizer.step()

    if batch % 400 == 0 :
      print(f"Looked At : {batch*len(X)}/{len(train_dataloader.dataset)} samples")

  train_loss = train_loss/len(train_dataloader)

  ##Testing LOOP

  test_loss , test_acc = 0 ,0
  model_0.eval()
  with torch.inference_mode():
    for X , y in test_dataloader:

      test_pred = model_0(X)

      test_loss = test_loss + loss_fn(test_pred , y)

      test_acc += accuracy_fn(y_true=y , y_pred= test_pred.argmax(dim=1))

    test_loss /= len(test_dataloader)

    test_acc /= len(test_dataloader)

  print(f"\nTrain Loss : {train_loss:.5f} | test Loss : {test_loss:.5f} | Test Acc : {test_acc:.2f}%\n")

train_end_time_cpu = timer()
total_train_time_cpu = print_train_time(start=train_start_time_cpu,
                                        end=train_end_time_cpu,
                                        device = str(next(model_0.parameters()).device))






In [ ]:
## Making Predictions From BaseLine Model i.e Model_0

In [ ]:
torch.manual_seed(42)

def eval_model(model : torch.nn.Module,
              data_loader : torch.utils.data.DataLoader,
              loss_fn : torch.nn.Module,
              accuracy_fn):

  loss , acc = 0 , 0
  model.eval()
  with torch.inference_mode():
    for X , y in data_loader:

      y_pred = model(X)

      loss += loss_fn(y_pred,y)
      acc += accuracy_fn(y_pred=y_pred.argmax(dim=1) ,
                         y_true = y )

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {"Model_Name" : model.__class__.__name__,
          "Model_Loss" : loss.item(),
          "Model_acc" : acc}

In [ ]:
model_0_results = eval_model(model=model_0,
                            data_loader=test_dataloader,
                            loss_fn = loss_fn,
                            accuracy_fn=accuracy_fn)

model_0_results

In [ ]:
## Lets Set Device Agnoistic Code and Make Use of GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
## Making The Model With NON-Linearlity
torch.manual_seed(42)

class FashionMNISTModelV1(nn.Module):
  def __init__(self,input_shape ,hidden_units , output_shape):
    super().__init__()

    self.Non_linearStack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=input_shape,
                  out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,
                  out_features=hidden_units),
        nn.ReLU(),
        nn.Linear(in_features=hidden_units,
                  out_features=output_shape),
        nn.ReLU()
    )

  def forward(self , x ):
    return self.Non_linearStack(x)

In [ ]:
model_1 = FashionMNISTModelV1(
    input_shape=28*28,
    hidden_units=16,
    output_shape=len(class_name)
).to(device)

model_1 , next(model_1.parameters()).device

In [ ]:
## Setting Loss And Optmizer Function
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr = 0.1)

In [ ]:
## Lets Functionise the Train And Test LOOP

In [ ]:
## Creating Train Loop Function
def Train_Step(model : torch.nn.Module,
               train_dataloader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               optimizer : torch.optim.Optimizer,
               accuracy_fn,
               device : torch.device = device):

  train_loss = 0
  train_acc = 0
  model.train()
  model.to(device)

  for batch , (X , y) in enumerate(train_dataloader):

    X, y = X.to(device) , y.to(device)

    y_pred = model(X)

    loss = loss_fn(y_pred , y)
    train_loss += loss

    acc = accuracy_fn(y_pred=y_pred.argmax(dim=1),
                        y_true = y)
    train_acc += acc


    optimizer.zero_grad()

    loss.backward()
    optimizer.step()

  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"Train_loss : {loss:.5f} | Train_Acc : {acc:.2f}%")


In [ ]:
## Creating Function for testing LOOP
def Test_Step(model : torch.nn.Module,
               test_dataloader : torch.utils.data.DataLoader,
               loss_fn : torch.nn.Module,
               accuracy_fn,
               device : torch.device = device):

  loss = 0
  acc = 0
  model.eval()
  model.to(device)

  with torch.inference_mode():
    for X , y in test_dataloader :

      X,y = X.to(device) , y.to(device)

      test_pred = model(X)

      test_loss = loss_fn(test_pred , y)
      loss += test_loss

      test_acc = accuracy_fn(y_pred=test_pred.argmax(dim=1),
                             y_true = y)
      acc += test_acc

  loss = loss/len(test_dataloader)
  acc = acc/len(test_dataloader)

  print(f"Test_loss : {loss:.5f} | Test_Acc : {acc:.2f}%\n")


In [ ]:
## Train and Test the Model

In [ ]:
# import tqdm
from tqdm.auto import tqdm

#setting seed and start time
torch.manual_seed(42)
train_start_time_gpu = timer()
# Set epochs
epochs = 3

##creating a loop
for epoch in tqdm(range(epochs)):
  print(f"Epoch : {epoch}\n------")
  ##Train Loop
  Train_Step(model = model_1 ,
             train_dataloader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device
             )


  ##Testing LOOP
  Test_Step(model=model_1,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             accuracy_fn=accuracy_fn,
             device = device
             )

train_end_time_gpu = timer()
total_train_time_gpu = print_train_time(start=train_start_time_gpu,
                                        end=train_end_time_gpu,
                                        device = device)


In [ ]:
model_0_results

In [ ]:
torch.manual_seed(42)

def eval_model(model : torch.nn.Module,
              data_loader : torch.utils.data.DataLoader,
              loss_fn : torch.nn.Module,
              accuracy_fn,
               device : torch.device = device):

  loss , acc = 0 , 0
  model.eval()
  model.to(device)
  with torch.inference_mode():
    for X , y in data_loader:
      X , y = X.to(device) , y.to(device)

      y_pred = model(X)

      loss += loss_fn(y_pred,y)
      acc += accuracy_fn(y_pred=y_pred.argmax(dim=1) ,
                         y_true = y )

    loss /= len(data_loader)
    acc /= len(data_loader)

  return {"Model_Name" : model.__class__.__name__,
          "Model_Loss" : loss.item(),
          "Model_acc" : acc}

In [ ]:
model_1_results = eval_model(model=model_1,
                             data_loader=test_dataloader,
                             accuracy_fn=accuracy_fn,
                             loss_fn=loss_fn,
                             device=device)

In [ ]:
model_1_results

**Learning And Building CNN Model**

In [ ]:
## Creating a CNN
class FashionMNiSTModelV2(nn.Module):
  def __init__(self , input_shape , hidden_units , output_shape):
    super().__init__()

    self.block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  padding=1,
                  stride=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=1)
    )

    self.block_2 = nn.Sequential(
        nn.Conv2d(hidden_units,hidden_units,3,padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units , hidden_units , 3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2,2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
    )
  def forward(self , x):
    x = self.block_1(x)
    # print(x.shape)
    x = self.block_2(x)
    # print(x.shape)
    x = self.classifier(x)
    # print(x.shape)

    return x

In [ ]:
torch.manual_seed(42)
model_2 = FashionMNiSTModelV2(input_shape=1,
                              hidden_units=10,
                              output_shape=len(class_name)).to(device)

model_2

In [ ]:
### Understanding The Conv2D and MaxPool2d Layers

In [ ]:
torch.manual_seed(42)

# Create sample batch of random numbers with same size as image batch
images = torch.randn(size=(32, 3, 64, 64)) # [batch_size, color_channels, height, width]
test_image = images[0] # get a single image for testing
print(f"Image batch shape: {images.shape} -> [batch_size, color_channels, height, width]")
print(f"Single image shape: {test_image.shape} -> [color_channels, height, width]")
print(f"Single image pixel values:\n{test_image}")

In [ ]:
torch.manual_seed(42)

# Create a convolutional layer with same dimensions as TinyVGG
# (try changing any of the parameters and see what happens)
conv_layer = nn.Conv2d(in_channels=3,
                       out_channels=10,
                       kernel_size=3,
                       stride=1,
                       padding=0) # also try using "valid" or "same" here

# Pass the data through the convolutional layer
conv_layer(test_image) # Note: If running PyTorch <1.11.0, this will error because of shape issues (nn.Conv.2d() expects a 4d tensor as input)

In [ ]:
# Add extra dimension to test image
test_image.unsqueeze(dim=0).shape

In [ ]:
# Pass test image with extra dimension through conv_layer
conv_layer(test_image.unsqueeze(dim=0)).shape

In [ ]:
# Print out original image shape without and with unsqueezed dimension
print(f"Test image original shape: {test_image.shape}")
print(f"Test image with unsqueezed dimension: {test_image.unsqueeze(dim=0).shape}")

# Create a sample nn.MaxPoo2d() layer
max_pool_layer = nn.MaxPool2d(kernel_size=2)

# Pass data through just the conv_layer
test_image_through_conv = conv_layer(test_image.unsqueeze(dim=0))
print(f"Shape after going through conv_layer(): {test_image_through_conv.shape}")

# Pass data through the max pool layer
test_image_through_conv_and_max_pool = max_pool_layer(test_image_through_conv)
print(f"Shape after going through conv_layer() and max_pool_layer(): {test_image_through_conv_and_max_pool.shape}")

In [ ]:
torch.manual_seed(42)
# Create a random tensor with a similar number of dimensions to our images
random_tensor = torch.randn(size=(1, 1, 2, 2))
print(f"Random tensor:\n{random_tensor}")
print(f"Random tensor shape: {random_tensor.shape}")

# Create a max pool layer
max_pool_layer = nn.MaxPool2d(kernel_size=2) # see what happens when you change the kernel_size value

# Pass the random tensor through the max pool layer
max_pool_tensor = max_pool_layer(random_tensor)
print(f"\nMax pool tensor:\n{max_pool_tensor} <- this is the maximum value from random_tensor")
print(f"Max pool tensor shape: {max_pool_tensor.shape}")

In [ ]:
## step up tha Optimizer and Loss Function
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model_2.parameters(),
                            lr = 0.1)

In [ ]:
## Training and Testing LOOP
torch.manual_seed(42)
torch.cuda.manual_seed(42)

#measure Time
from timeit import default_timer as timer
train_start_time_model_2 = timer()

epochs = 3

for epoch in tqdm(range(epochs)):
  print(f"Epoch : {epoch}\n ---")

  Train_Step(model= model_2,
             train_dataloader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device = device)

  Test_Step(model=model_2,
            test_dataloader=test_dataloader,
            accuracy_fn=accuracy_fn,
            loss_fn=loss_fn,
            device = device)


train_end_time_model_2 = timer()
total_train_time_model_2 = print_train_time(start=train_start_time_model_2,
                                            end=train_end_time_model_2,
                                            device = device
                                            )


In [ ]:
## model_2 results
model_2_results = eval_model(
    model=model_2,
    accuracy_fn=accuracy_fn,
    data_loader=test_dataloader,
    loss_fn=loss_fn,
    device = device
)
model_2_results

In [ ]:
## Lest Compare
import pandas as pd

compare_results = pd.DataFrame([model_0_results,model_1_results,model_2_results])
compare_results

In [ ]:
compare_results["train_time"] = [total_train_time_cpu ,total_train_time_gpu,total_train_time_model_2]
compare_results

In [ ]:
##Make and Evaluate Random Predictions with Best Model

def make_predictions(model : nn.Module,
                     data : list ,
                     device : torch.device = device):
  pred_probs =[]
  model.to(device)
  model.eval()
  with torch.inference_mode():
    for sample in data :

      sample = torch.unsqueeze(sample , dim = 0).to(device)

      pred_logits = model(sample)

      pred_prob = torch.softmax(pred_logits.squeeze() , dim = 0 )

      pred_probs.append(pred_prob.cpu())

  return torch.stack(pred_probs)


In [ ]:
import random
# random.seed(42)

test_samples = []
test_labels = []

for sample , label in random.sample(list(test_data) , k = 9 ):
  test_samples.append(sample)
  test_labels.append(label)

test_samples[0].shape



In [ ]:
plt.imshow(test_samples[0].squeeze() , cmap="gray")
plt.title(class_name[test_labels[0]])

In [ ]:
## Make predictions
pred_prods = make_predictions(model = model_2,
                              data = test_samples)

In [ ]:
pred_prods[:4]

In [ ]:
## Probs to labels
pred_classes = pred_prods.argmax(dim=1)
pred_classes

In [ ]:
test_labels

In [ ]:
## Plot predictions
plt.figure(figsize=(9,9))
row = 3
col = 3

for i , sample in enumerate(test_samples):
  plt.subplot(row,col,i+1)

  plt.imshow(sample.squeeze(),cmap= "gray")

  pred_label = class_name[pred_classes[i]]
  truth_label = class_name[test_labels[i]]

  title_text = f"Pred : {pred_label} | Truth : {truth_label}"

  if pred_label == truth_label:
    plt.title(title_text , c = "g" ,fontsize = 10 )
  else: plt.title(title_text , c="r" , fontsize = 10)

  plt.axis(False)



In [ ]:
# Import tqdm for progress bar
from tqdm.auto import tqdm

# 1. Make predictions with trained model
y_preds = []
model_2.eval()
with torch.inference_mode():
  for X, y in tqdm(test_dataloader, desc="Making predictions"):
    # Send data and targets to target device
    X, y = X.to(device), y.to(device)
    # Do the forward pass
    y_logit = model_2(X)
    # Turn predictions from logits -> prediction probabilities -> predictions labels
    y_pred = torch.softmax(y_logit, dim=1).argmax(dim=1) # note: perform softmax on the "logits" dimension, not "batch" dimension (in this case we have a batch size of 32, so can perform on dim=1)
    # Put predictions on CPU for evaluation
    y_preds.append(y_pred.cpu())
# Concatenate list of predictions into a tensor
y_pred_tensor = torch.cat(y_preds)

In [ ]:
y_pred_tensor.size()

In [ ]:
# Import mlxtend upgraded version
import mlxtend
print(mlxtend.__version__)
assert int(mlxtend.__version__.split(".")[1]) >= 19 # should be version 0.19.0 or higher


In [ ]:
!pip install torchmetrics

In [ ]:
len(class_name)

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

confmat = ConfusionMatrix(num_classes=len(class_name),task='multiclass')
confmat_tensor = confmat(preds = y_pred_tensor,
                         target = test_data.targets)

fig , ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(),
    class_names=class_name,
    figsize=(8,5)
)

In [ ]:
pip install nbstripout